# 🔍 Análise de Problemas de Qualidade - TechCommerce

Este notebook identifica e classifica todos os problemas de qualidade nos datasets da TechCommerce.

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, date
import warnings
warnings.filterwarnings('ignore')

# Configuração para exibir todas as colunas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 📊 Carregamento dos Datasets

In [ ]:
# Carregamento dos datasets
clientes_df = pd.read_csv('../data/raw/clientes.csv')
produtos_df = pd.read_csv('../data/raw/produtos.csv')
vendas_df = pd.read_csv('../data/raw/vendas.csv')
logistica_df = pd.read_csv('../data/raw/logistica.csv')

print("📊 Datasets carregados:")
print(f"Clientes: {len(clientes_df)} registros")
print(f"Produtos: {len(produtos_df)} registros")
print(f"Vendas: {len(vendas_df)} registros")
print(f"Logística: {len(logistica_df)} registros")

## 🔍 Análise de Problemas por Dataset

In [ ]:
def analisar_problemas_clientes(df):
    problemas = []
    total_registros = len(df)
    
    # 1. COMPLETUDE
    nulos_nome = df['nome'].isnull().sum()
    nulos_email = df['email'].isnull().sum()
    vazios_nome = (df['nome'] == '').sum()
    
    if nulos_nome > 0 or vazios_nome > 0:
        problemas.append({
            'dataset': 'clientes',
            'dimensao': 'Completude',
            'problema': 'Nomes nulos ou vazios',
            'registros_afetados': nulos_nome + vazios_nome,
            'percentual': ((nulos_nome + vazios_nome) / total_registros) * 100,
            'criticidade': 'Crítico'
        })
    
    if nulos_email > 0:
        problemas.append({
            'dataset': 'clientes',
            'dimensao': 'Completude',
            'problema': 'Emails nulos',
            'registros_afetados': nulos_email,
            'percentual': (nulos_email / total_registros) * 100,
            'criticidade': 'Alto'
        })
    
    # 2. UNICIDADE
    duplicatas_id = df['id_cliente'].duplicated().sum()
    if duplicatas_id > 0:
        problemas.append({
            'dataset': 'clientes',
            'dimensao': 'Unicidade',
            'problema': 'IDs duplicados',
            'registros_afetados': duplicatas_id,
            'percentual': (duplicatas_id / total_registros) * 100,
            'criticidade': 'Crítico'
        })
    
    # 3. VALIDADE
    emails_invalidos = 0
    telefones_invalidos = 0
    
    for idx, row in df.iterrows():
        if pd.notna(row['email']) and not re.match(r'^[\w\.-]+@[\w\.-]+\.\w+$', str(row['email'])):
            emails_invalidos += 1
        
        if pd.notna(row['telefone']) and not re.match(r'^[1-9][1-9][0-9]{9}$', str(row['telefone'])):
            telefones_invalidos += 1
    
    if emails_invalidos > 0:
        problemas.append({
            'dataset': 'clientes',
            'dimensao': 'Validade',
            'problema': 'Emails com formato inválido',
            'registros_afetados': emails_invalidos,
            'percentual': (emails_invalidos / total_registros) * 100,
            'criticidade': 'Alto'
        })
    
    if telefones_invalidos > 0:
        problemas.append({
            'dataset': 'clientes',
            'dimensao': 'Validade',
            'problema': 'Telefones com formato inválido',
            'registros_afetados': telefones_invalidos,
            'percentual': (telefones_invalidos / total_registros) * 100,
            'criticidade': 'Médio'
        })
    
    return problemas

# Análise dos clientes
problemas_clientes = analisar_problemas_clientes(clientes_df)
print("🔍 Problemas encontrados em CLIENTES:")
for p in problemas_clientes:
    print(f"  • {p['problema']}: {p['registros_afetados']} registros ({p['percentual']:.1f}%) - {p['criticidade']}")

In [ ]:
def analisar_problemas_produtos(df):
    problemas = []
    total_registros = len(df)
    
    # 1. COMPLETUDE
    nulos_categoria = df['categoria'].isnull().sum()
    vazios_categoria = (df['categoria'] == '').sum()
    
    if nulos_categoria > 0 or vazios_categoria > 0:
        problemas.append({
            'dataset': 'produtos',
            'dimensao': 'Completude',
            'problema': 'Categorias nulas ou vazias',
            'registros_afetados': nulos_categoria + vazios_categoria,
            'percentual': ((nulos_categoria + vazios_categoria) / total_registros) * 100,
            'criticidade': 'Alto'
        })
    
    # 2. UNICIDADE
    duplicatas_id = df['id_produto'].duplicated().sum()
    if duplicatas_id > 0:
        problemas.append({
            'dataset': 'produtos',
            'dimensao': 'Unicidade',
            'problema': 'IDs duplicados',
            'registros_afetados': duplicatas_id,
            'percentual': (duplicatas_id / total_registros) * 100,
            'criticidade': 'Crítico'
        })
    
    # Duplicatas de produto (mesmo nome + categoria)
    duplicatas_produto = df.duplicated(subset=['nome_produto', 'categoria']).sum()
    if duplicatas_produto > 0:
        problemas.append({
            'dataset': 'produtos',
            'dimensao': 'Unicidade',
            'problema': 'Produtos duplicados (nome + categoria)',
            'registros_afetados': duplicatas_produto,
            'percentual': (duplicatas_produto / total_registros) * 100,
            'criticidade': 'Alto'
        })
    
    # 3. VALIDADE
    precos_negativos = (df['preco'] < 0).sum()
    estoque_negativo = (df['estoque'] < 0).sum()
    
    if precos_negativos > 0:
        problemas.append({
            'dataset': 'produtos',
            'dimensao': 'Validade',
            'problema': 'Preços negativos',
            'registros_afetados': precos_negativos,
            'percentual': (precos_negativos / total_registros) * 100,
            'criticidade': 'Crítico'
        })
    
    if estoque_negativo > 0:
        problemas.append({
            'dataset': 'produtos',
            'dimensao': 'Validade',
            'problema': 'Estoque negativo',
            'registros_afetados': estoque_negativo,
            'percentual': (estoque_negativo / total_registros) * 100,
            'criticidade': 'Alto'
        })
    
    return problemas

# Análise dos produtos
problemas_produtos = analisar_problemas_produtos(produtos_df)
print("\n🔍 Problemas encontrados em PRODUTOS:")
for p in problemas_produtos:
    print(f"  • {p['problema']}: {p['registros_afetados']} registros ({p['percentual']:.1f}%) - {p['criticidade']}")

In [ ]:
def analisar_problemas_vendas(df, clientes_df, produtos_df):
    problemas = []
    total_registros = len(df)
    
    # 1. VALIDADE
    quantidades_negativas = (df['quantidade'] <= 0).sum()
    valores_negativos = (df['valor_total'] < 0).sum()
    
    if quantidades_negativas > 0:
        problemas.append({
            'dataset': 'vendas',
            'dimensao': 'Validade',
            'problema': 'Quantidades negativas ou zero',
            'registros_afetados': quantidades_negativas,
            'percentual': (quantidades_negativas / total_registros) * 100,
            'criticidade': 'Crítico'
        })
    
    if valores_negativos > 0:
        problemas.append({
            'dataset': 'vendas',
            'dimensao': 'Validade',
            'problema': 'Valores totais negativos',
            'registros_afetados': valores_negativos,
            'percentual': (valores_negativos / total_registros) * 100,
            'criticidade': 'Crítico'
        })
    
    # 2. INTEGRIDADE REFERENCIAL
    clientes_validos = set(clientes_df['id_cliente'].unique())
    produtos_validos = set(produtos_df['id_produto'].unique())
    
    clientes_orfaos = df[~df['id_cliente'].isin(clientes_validos)].shape[0]
    produtos_orfaos = df[~df['id_produto'].isin(produtos_validos)].shape[0]
    
    if clientes_orfaos > 0:
        problemas.append({
            'dataset': 'vendas',
            'dimensao': 'Integridade',
            'problema': 'Referências órfãs para clientes',
            'registros_afetados': clientes_orfaos,
            'percentual': (clientes_orfaos / total_registros) * 100,
            'criticidade': 'Crítico'
        })
    
    if produtos_orfaos > 0:
        problemas.append({
            'dataset': 'vendas',
            'dimensao': 'Integridade',
            'problema': 'Referências órfãs para produtos',
            'registros_afetados': produtos_orfaos,
            'percentual': (produtos_orfaos / total_registros) * 100,
            'criticidade': 'Crítico'
        })
    
    # 3. CONSISTÊNCIA
    inconsistencias_calculo = 0
    datas_futuras = 0
    
    for idx, row in df.iterrows():
        # Verificar cálculo valor_total
        valor_esperado = row['quantidade'] * row['valor_unitario']
        if abs(row['valor_total'] - valor_esperado) > 0.01:  # tolerância de 1 centavo
            inconsistencias_calculo += 1
        
        # Verificar datas futuras
        try:
            data_venda = pd.to_datetime(row['data_venda']).date()
            if data_venda > date.today():
                datas_futuras += 1
        except:
            pass
    
    if inconsistencias_calculo > 0:
        problemas.append({
            'dataset': 'vendas',
            'dimensao': 'Consistência',
            'problema': 'Inconsistência no cálculo valor_total',
            'registros_afetados': inconsistencias_calculo,
            'percentual': (inconsistencias_calculo / total_registros) * 100,
            'criticidade': 'Alto'
        })
    
    if datas_futuras > 0:
        problemas.append({
            'dataset': 'vendas',
            'dimensao': 'Atualidade',
            'problema': 'Datas de venda futuras',
            'registros_afetados': datas_futuras,
            'percentual': (datas_futuras / total_registros) * 100,
            'criticidade': 'Crítico'
        })
    
    return problemas

# Análise das vendas
problemas_vendas = analisar_problemas_vendas(vendas_df, clientes_df, produtos_df)
print("\n🔍 Problemas encontrados em VENDAS:")
for p in problemas_vendas:
    print(f"  • {p['problema']}: {p['registros_afetados']} registros ({p['percentual']:.1f}%) - {p['criticidade']}")

In [ ]:
def analisar_problemas_logistica(df, vendas_df):
    problemas = []
    total_registros = len(df)
    
    # 1. COMPLETUDE
    nulos_transportadora = df['transportadora'].isnull().sum()
    vazios_transportadora = (df['transportadora'] == '').sum()
    nulos_data_envio = df['data_envio'].isnull().sum()
    
    if nulos_transportadora > 0 or vazios_transportadora > 0:
        problemas.append({
            'dataset': 'logistica',
            'dimensao': 'Completude',
            'problema': 'Transportadora nula ou vazia',
            'registros_afetados': nulos_transportadora + vazios_transportadora,
            'percentual': ((nulos_transportadora + vazios_transportadora) / total_registros) * 100,
            'criticidade': 'Alto'
        })
    
    if nulos_data_envio > 0:
        problemas.append({
            'dataset': 'logistica',
            'dimensao': 'Completude',
            'problema': 'Data de envio nula',
            'registros_afetados': nulos_data_envio,
            'percentual': (nulos_data_envio / total_registros) * 100,
            'criticidade': 'Alto'
        })
    
    # 2. INTEGRIDADE REFERENCIAL
    vendas_validas = set(vendas_df['id_venda'].unique())
    vendas_orfaos = df[~df['id_venda'].isin(vendas_validas)].shape[0]
    
    if vendas_orfaos > 0:
        problemas.append({
            'dataset': 'logistica',
            'dimensao': 'Integridade',
            'problema': 'Referências órfãs para vendas',
            'registros_afetados': vendas_orfaos,
            'percentual': (vendas_orfaos / total_registros) * 100,
            'criticidade': 'Crítico'
        })
    
    # 3. CONSISTÊNCIA
    inconsistencias_datas = 0
    
    for idx, row in df.iterrows():
        try:
            if pd.notna(row['data_envio']) and pd.notna(row['data_entrega_real']):
                data_envio = pd.to_datetime(row['data_envio'])
                data_entrega = pd.to_datetime(row['data_entrega_real'])
                if data_entrega < data_envio:
                    inconsistencias_datas += 1
        except:
            pass
    
    if inconsistencias_datas > 0:
        problemas.append({
            'dataset': 'logistica',
            'dimensao': 'Consistência',
            'problema': 'Data entrega anterior ao envio',
            'registros_afetados': inconsistencias_datas,
            'percentual': (inconsistencias_datas / total_registros) * 100,
            'criticidade': 'Alto'
        })
    
    return problemas

# Análise da logística
problemas_logistica = analisar_problemas_logistica(logistica_df, vendas_df)
print("\n🔍 Problemas encontrados em LOGÍSTICA:")
for p in problemas_logistica:
    print(f"  • {p['problema']}: {p['registros_afetados']} registros ({p['percentual']:.1f}%) - {p['criticidade']}")

## 📊 Consolidação e Priorização

In [ ]:
# Consolidar todos os problemas
todos_problemas = problemas_clientes + problemas_produtos + problemas_vendas + problemas_logistica

# Criar DataFrame para análise
df_problemas = pd.DataFrame(todos_problemas)

# Ordenar por criticidade e percentual
ordem_criticidade = {'Crítico': 4, 'Alto': 3, 'Médio': 2, 'Baixo': 1}
df_problemas['ordem_criticidade'] = df_problemas['criticidade'].map(ordem_criticidade)
df_problemas = df_problemas.sort_values(['ordem_criticidade', 'percentual'], ascending=[False, False])

print("📊 RESUMO EXECUTIVO - Problemas de Qualidade TechCommerce")
print("=" * 70)

# Estatísticas gerais
total_problemas = len(df_problemas)
problemas_criticos = len(df_problemas[df_problemas['criticidade'] == 'Crítico'])
problemas_altos = len(df_problemas[df_problemas['criticidade'] == 'Alto'])

print(f"\n🚨 TOTAL DE PROBLEMAS IDENTIFICADOS: {total_problemas}")
print(f"   • Críticos: {problemas_criticos}")
print(f"   • Altos: {problemas_altos}")
print(f"   • Médios: {len(df_problemas[df_problemas['criticidade'] == 'Médio'])}")

print("\n🎯 TOP 10 PROBLEMAS PRIORITÁRIOS:")
for idx, row in df_problemas.head(10).iterrows():
    print(f"   {idx+1:2d}. [{row['dataset'].upper()}] {row['problema']}")
    print(f"       Dimensão: {row['dimensao']} | Impacto: {row['percentual']:.1f}% | Criticidade: {row['criticidade']}")
    print()

# Análise por dimensão
print("\n📊 PROBLEMAS POR DIMENSÃO DA QUALIDADE:")
dimensoes = df_problemas.groupby('dimensao').agg({
    'problema': 'count',
    'registros_afetados': 'sum',
    'percentual': 'mean'
}).round(1)
dimensoes.columns = ['Qtd_Problemas', 'Total_Registros_Afetados', 'Percentual_Medio']
print(dimensoes.sort_values('Qtd_Problemas', ascending=False))

# Análise por dataset
print("\n📊 PROBLEMAS POR DATASET:")
datasets = df_problemas.groupby('dataset').agg({
    'problema': 'count',
    'registros_afetados': 'sum',
    'percentual': 'mean'
}).round(1)
datasets.columns = ['Qtd_Problemas', 'Total_Registros_Afetados', 'Percentual_Medio']
print(datasets.sort_values('Qtd_Problemas', ascending=False))

## 💾 Exportar Resultados

In [ ]:
# Salvar análise completa
df_problemas.drop('ordem_criticidade', axis=1).to_csv('../data/quality/analise_problemas_completa.csv', index=False)

# Criar relatório executivo
relatorio = f"""
# 📊 Relatório Executivo - Análise de Qualidade de Dados TechCommerce

## Resumo Executivo
- **Total de problemas identificados**: {total_problemas}
- **Problemas críticos**: {problemas_criticos} (requerem ação imediata)
- **Problemas de alta prioridade**: {problemas_altos}

## Principais Achados
1. **Integridade Referencial**: Vendas com referências órfãs para clientes inexistentes
2. **Validade de Dados**: Preços negativos e quantidades inválidas
3. **Completude**: Campos obrigatórios vazios em múltiplos datasets
4. **Unicidade**: Duplicatas em chaves primárias

## Recomendações Imediatas
1. Implementar validações de integridade referencial
2. Corrigir valores negativos em preços e quantidades
3. Estabelecer processo de deduplicação
4. Implementar validação de formatos (emails, telefones)

## Impacto no Negócio
- **Receita**: Vendas com valores negativos impactam faturamento
- **Operações**: Referências órfãs impedem processamento correto
- **Compliance**: Dados incompletos violam políticas de governança
- **Decisões**: Dados inconsistentes comprometem análises
"""

with open('../data/quality/relatorio_executivo_problemas.md', 'w', encoding='utf-8') as f:
    f.write(relatorio)

print("✅ Análise completa salva em:")
print("   • ../data/quality/analise_problemas_completa.csv")
print("   • ../data/quality/relatorio_executivo_problemas.md")